In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select

import time
import os
import zipfile

import shutil
from glob import glob
import pandas as pd
from datetime import datetime


In [2]:
# Папка, где хранится трекер
download_dir = r"C:\Users\tatarikov\Desktop\R scripts\Daily report"

In [3]:
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

In [4]:
# Здесь был переход по ссылке с ИРК

In [5]:
wait = WebDriverWait(driver, 10)
input_field = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "#id_username"
    )))
input_field.send_keys("user")

In [6]:
input_field = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "#id_password"
    )))
input_field.send_keys("password")

In [7]:
button = driver.find_element(By.CSS_SELECTOR, "#submit-id-submit")
button.click()

In [ ]:
# Здесь был переход в раздел с находками

In [9]:
# Находим поле ввода
input_field = driver.find_element(By.CSS_SELECTOR, "#wrap > div.container-fluid.main > main > query-tbl > table > thead > tr.ng-table-filters > th:nth-child(2) > div > input")

# Очистка поля 
input_field.clear()

In [ ]:
import pandas as pd
from datetime import datetime
import os

# Установка рабочей директории
work_dir = r"C:\Users\tatarikov\Desktop\R scripts\Daily report"
os.chdir(work_dir)

# Получение сегодняшней даты в формате ДД.ММ.ГГГГ
today_str = datetime.today().strftime('%d.%m.%Y')

# Формирование имени файла
filename = f"not_yes_queries.xlsx"

# Чтение Excel-файла
filtered_df = pd.read_excel(filename, engine='openpyxl', skiprows=0)

# Вывод первых строк
print(filtered_df.head())

In [11]:
# Вывод количества строк
print(len(filtered_df))

50


In [ ]:
# === Обработка каждой строки ===
for index, row in filtered_df.iterrows():
    query_number = row["Query.number"]
    print(f"\n=== Обработка запроса [{query_number}] ===")

    try:
        # Вводим Query.number в фильтр
        input_field = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > thead > tr.ng-table-filters > th:nth-child(4) input")
        input_field.clear()
        input_field.send_keys(str(query_number))
        input_field.send_keys(Keys.ENTER)
        time.sleep(2)

        # Определяем тип запроса
        type_field = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(5)")
        type_value = type_field.text.strip()
        print(f"[{query_number}] Тип: {type_value}")

        # Нажимаем кнопку "Edit"
        action_button = driver.find_element(By.CSS_SELECTOR,
            "#wrap > div.container-fluid.main > main > query-tbl > table > tbody > tr > td:nth-child(13) > span:nth-child(2) > button")
        action_button.click()
        time.sleep(2)

        # === Изменение статуса на "New" ===
        # Находим сначала саму модалку
        modal = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.modal.fade.in"))
        )
        time.sleep(1)
        # Затем уже ищем select внутри модалки
        status_select_element = modal.find_element(By.NAME, "status")
        
        # Выбираем нужное значение
        select = Select(status_select_element)
        select.select_by_visible_text("New")
        print(f"[{query_number}] Выбрано: New.")
        time.sleep(1)
        # === Если тип — Query monitoring data ===
        if type_value == "Query monitoring data":
            # Изменяем Description на "Query response monitored"
            time.sleep(2)
            description_wrapper = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.col-sm-9[ng-if='qrAeVm.edit']"))
            )
            description_select = Select(description_wrapper.find_element(By.NAME, "description"))
            description_select.select_by_visible_text("Incorrect data")
            print(f"[{query_number}] Description установлен: Incorrect data")

        # === Вводим комментарий ===
        textarea = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@ng-if, 'qrAeVm.queryIsNotAnnotationAndClosed')]//textarea[@name='detail_note']"))
        )

        comment_text = "Please enter the analysis data in this eCRF form regardless of whether it was performed in the central or local laboratory"
        driver.execute_script("""
            var el = arguments[0];
            var text = arguments[1];
            el.value = text;
            var event = new Event('input', { bubbles: true });
            el.dispatchEvent(event);
            var changeEvent = new Event('change', { bubbles: true });
            el.dispatchEvent(changeEvent);
        """, textarea, comment_text)
        print(f"[{query_number}] Комментарий добавлен.")

        # === Сохраняем изменения ===
        save_button = driver.find_element(By.CSS_SELECTOR,
            "body > div.modal.fade.in > div > div > div.modal-footer > button.btn.btn-primary")
        save_button.click()
        print(f"[{query_number}] Сохранение выполнено.")

        # Обновляем статус в DataFrame
        filtered_df.at[index, 'Status'] = 'Closed'

    except Exception as e:
        print(f"[{query_number}] Ошибка: {e}")

    time.sleep(2)


=== Обработка запроса [763237] ===
[763237] Тип: Query monitoring data
[763237] Выбрано: New.
[763237] Description установлен: Incorrect data
[763237] Комментарий добавлен.
[763237] Сохранение выполнено.

=== Обработка запроса [763278] ===
[763278] Тип: Query monitoring data
[763278] Выбрано: New.
[763278] Description установлен: Incorrect data
[763278] Комментарий добавлен.
[763278] Сохранение выполнено.

=== Обработка запроса [762870] ===
[762870] Тип: Query monitoring data
[762870] Выбрано: New.
[762870] Description установлен: Incorrect data
[762870] Комментарий добавлен.
[762870] Сохранение выполнено.

=== Обработка запроса [762930] ===
[762930] Тип: Reason for change
[762930] Выбрано: New.
[762930] Комментарий добавлен.
[762930] Сохранение выполнено.

=== Обработка запроса [762932] ===
[762932] Тип: Reason for change
[762932] Выбрано: New.
[762932] Комментарий добавлен.
[762932] Сохранение выполнено.

=== Обработка запроса [763413] ===
[763413] Тип: Query monitoring data
[763413